In [1]:
from IPython.core.display import HTML

In [2]:
url_src = "http://labrosa.ee.columbia.edu/millionsong/sites/default/files/tutorial4.pdf"
HTML("<iframe src='%s' width=1000 height=1000></iframe>"%url_src)

In [40]:
import os
import sys
import glob
import time
import datetime
import numpy as np
try:
    import sqlite3
except ImportError:
    print ' you need sqlite3 installed to use this program'
    sys.exit(0)
    
def encode_string(s):
    """
    Simple utility function to make sure a string is proper
    to be used in a SQLite query
    (different than postgresql, no N to specify unicode)
    EXAMPLE:
      That's my boy! -> 'That''s my boy!'
    """
    return "'" + s.replace("'", "''") + "'"

In [9]:
# PATH TO artist_term.db
# CHANGE THIS TO YOUR LOCAL CONFIGURATION
# IT SHOULD BE IN THE ADDITIONAL FILES
# (you can use 'subset_artist_term.db')
dbfile = '/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/subset_artist_term.db'

In [13]:
# connect to sqlite3 database
conn = sqlite3.connect(dbfile)

# from that connection get cursor to do queries
c = conn.cursor()

# List Tables in Schema/DB

In [11]:
### schema overview
# we got 3 tables:
# table1: name=artists #cols=1 (artist_id text)
#      one row per artist, no duplicates, usually in alphabetical order
# table2: name=terms   #cols=1 (term text)
#      one row per term, no duplicates, usually alphabetical order
# table3: name=artist_term #cols2 (artist_id text, term text)
#      One row per pair artist_id/term, no duplicate pairs
#      Entries in table3 are constrained by table1 and table2,
# e.g. artist_id must exist in table1 before it can be used in table3
# NOT ALL ARTISTS HAVE TERMS. They will still all be in table1, but
# some artists are not in table3 at all

print '*************** GENERAL SQLITE DEMO ***************'

*************** GENERAL SQLITE DEMO ***************


In [14]:
# list all tables in the dataset
# not that sqlite does the actual job when we call fetchone() or fetchall()
q = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name"
res = c.execute(q)
print "* tables contained in the SQLite file/database (there should be 3):"

* tables contained in the SQLite file/database (there should be 3):


In [15]:
print res.fetchall()

[(u'artist_mbtag',), (u'artist_term',), (u'artists',), (u'mbtags',), (u'terms',)]


In [16]:
# list all indices
q = "SELECT name FROM sqlite_master WHERE type='index' ORDER BY name"
res = c.execute(q)
print "* indices in the database to make reads faster"

* indices in the database to make reads faster


In [17]:
print res.fetchall()

[(u'idx_artist_id_mbtag',), (u'idx_artist_id_term',), (u'idx_mbtag_artist_id',), (u'idx_term_artist_id',), (u'sqlite_autoindex_artists_1',), (u'sqlite_autoindex_mbtags_1',), (u'sqlite_autoindex_terms_1',)]


# List ARTISTS

In [18]:
print "*************** ARTISTS TABLE DEMO ****************************"

*************** ARTISTS TABLE DEMO ****************************


In [19]:
# list all artists
q = "SELECT * FROM artists"
res = c.execute(q)
print "* list all known artists in the database (display the first 3)"

* list all known artists in the database (display the first 3)


In [20]:
print res.fetchall()[:3]

[(u'AR009211187B989185',), (u'AR00A6H1187FB5402A',), (u'AR00LNI1187FB444A5',)]


In [21]:
# list all artists that id starts with ARB
q = "SELECT artist_id FROM artists WHERE SUBSTR(artist_id, 1, 3)='ARB' LIMIT 2"
res = c.execute(q)
print "* list all artists whose ID starts with ARB (we only ask for the first 2"

* list all artists whose ID starts with ARB (we only ask for the first 2


In [22]:
print res.fetchall()

[(u'ARB054P1187B9AD32E',), (u'ARB06Z71187B99049A',)]


# Count all Artists in the Artists Table

In [23]:
# count all artists
q = "SELECT COUNT(artist_id) FROM artists"
res = c.execute(q)
print "* count the number of artists (with or without tags):"

* count the number of artists (with or without tags):


In [24]:
print res.fetchone()

(3888,)


# Show Terms from Terms Table

In [25]:
print "*************** TERMS TABLE ***************"

*************** TERMS TABLE ***************


In [26]:
# list all terms (=tags)
q = "SELECT * FROM terms"
res = c.execute(q)
print "* list all known terms in the database (display the first 3):"

* list all known terms in the database (display the first 3):


In [27]:
print res.fetchall()[:3]

[(u'00s',), (u'00s country',), (u'00s pop',)]


In [28]:
# list all terms that start with indie
q = "SELECT term FROM terms WHERE SUBSTR(term, 1, 5)='indie' LIMIT 3"
res = c.execute(q)
print "* list terms that start with 'indie' (we ask for 3 of them):"

* list terms that start with 'indie' (we ask for 3 of them):


In [29]:
print res.fetchall()

[(u'indie',), (u'indie acoustic',), (u'indie artist',)]


In [31]:
q1 = "SELECT term FROM terms WHERE term='rock' LIMIT 1"
q2 = "SELECT term FROM terms WHERE term='abc123xyz'"
res = c.execute(q1)
res1_str = str(res.fetchone())
res = c.execute(q2)
res2_str = str(res.fetchone())
print "* we check if two tags are in the same database, (the first one, is):"

* we check if two tags are in the same database, (the first one, is):


In [32]:
print 'rock:' , res1_str, ', abc123xyz:', res2_str

rock: (u'rock',) , abc123xyz: None


In [33]:
# similar for mtags, list all mbtags
q = "SELECT * FROM mbtags"
res = c.execute(q)
print "btags work the same as terms, e.g., list all known mbtags (display the first 3):"

btags work the same as terms, e.g., list all known mbtags (display the first 3):


In [34]:
print res.fetchall()[:3]

[(u'00s',), (u'1 13 165900 150 7672 22647 34612 48720 59280 74602 87545 95495 107182 131087 141522 153710',), (u'1 7 186240 183 23558 41608 89158 111733 150833 169883',)]


# ARTIST-TERM DEMO

In [35]:
print "************** ARTIST / TERM DEMO ***************"

************** ARTIST / TERM DEMO ***************


## get tags given an artist_id

In [37]:
# get all the tags from "The Beatles"
q = "SELECT term FROM artist_term WHERE artist_id='AR6XZ861187FB4CECD'"
res = c.execute(q)
print "* we get all tags from The Beatles (since we know their id) and we show 4"
print res.fetchall()[:4]

* we get all tags from The Beatles (since we know their id) and we show 4
[(u'60s',), (u'acoustic',), (u'am pop',), (u'ambient',)]


## count number of unique tags

In [38]:
# count the number of tags applied to the Beatles
q = "SELECT COUNT(term) FROM artist_term WHERE artist_id='AR6XZ861187FB4CECD'"
res = c.execute(q)
print "* we count the number of unique tags applied to The Beatles"
print res.fetchone()

* we count the number of unique tags applied to The Beatles
(30,)


## count artist ids with tag "jazz"

In [42]:
# get artist ids that have been tagged with 'jazz'
# note the encode_string function, which mostly doubles the '' sign
q = "SELECT artist_id FROM artist_term WHERE term=" + encode_string('jazz')
q += " ORDER BY RANDOM() LIMIT 2"
res = c.execute(q)
print "* we get all artists tagged with jazz (we display 2 at random):"
print res.fetchall()

* we get all artists tagged with jazz (we display 2 at random):
[(u'AR7S48C1187B99711E',), (u'ARH3TXI1187FB3A486',)]


In [44]:
q = "SELECT COUNT(artist_id) FROM artist_term WHERE term=" + encode_string('rock')
res = c.execute(q)
print "* we got the number of terms that got the term 'rock':"
print res.fetchone()

* we got the number of terms that got the term 'rock':
(2346,)


## differentiate between tags and mbtags

In [45]:
# count the number of artists that have the term 'rock' but not the mbtag 'rock'
q = "SELECT artist_id FROM artist_term WHERE term=" + encode_string('rock')
q += " EXCEPT SELECT artist_id FROM artist_mbtag WHERE mbtag=" + encode_string('rock')
q += " LIMIT 1"

res = c.execute(q)
print "* one artist has the tag 'rock' but not mbtag 'rock'"
print res.fetchone()

* one artist has the tag 'rock' but not mbtag 'rock'
(u'AR009211187B989185',)


## artists with no terms

In [46]:
q = "SELECT artist_id FROM artists EXCEPT SELECT artist_id FROM artist_term LIMIT 1"
res = c.execute(q)
artist_notag = res.fetchone()
print "* we show an artist with no terms:"
if artist_notag is None:
    # check if artists have at least one tag, can be slow
    q = "SELECT * FROM artists"
    res = c.execute(q)
    allartists = map(lambda x : x[0], res.fetchall())
    for art in allartists:
        q = "SELECT COUNT(term) FROM artist_term WHERE artist_id='" + art + "'"
        res.execute(q)
        assert res.fetchone()[0] > 0
    print '(found no artist with no terms, we double-checked)'
else:
    print artist_notag

* we show an artist with no terms:
(u'AR2D6TV1187B9B2FBF',)


## close up the connection

In [47]:
c.close()
conn.close()